In [ ]:
import numpy as np
import random as rd
from scipy.stats import norm
from scipy.optimize import minimize_scalar  
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os
import pandas as pd 
import datetime as dt
import scipy as sp
import matplotlib.mlab as mlab
import pandas as pd 
import statistics as stats
import seaborn as sns
import math
import matplotlib.dates as mdates
import mpmath
from scipy.integrate import quad 
import scipy.optimize as optimize
import time
from fbm import FBM
from timeit import default_timer as timer



from scipy.stats import norm
from scipy.special import gamma
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import moment 
from sympy import *


In [ ]:
rd.seed(10)
sns.set()

__Parameters__

In [ ]:
T = 200
n = 6 #must be an integer 
N = T*n
delta = T/N 
steps = N
r = 0.02

__Real data__

In [ ]:
data = pd.read_excel(
     os.path.join("realisedvariancesourceredclosingprice.xlsx"),
     engine='openpyxl',
)
list_dates_formated = []
list_dates = list(data['Unnamed: 0'])
list_names = list(data['Symbol'])
list_closing_prices= list(data['close_price'])
A_all = []
size_current = len(list_dates)
for i in range(0,size_current):
    date = list_dates[i].split(" ")[0].split("-")
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])    
    list_dates_formated.append(dt.datetime(year,month,day))
    A_all.append([dt.datetime(year,month,day),list_names[i],list_closing_prices[i]])
df_all_returns = pd.DataFrame(A_all)
df_all_returns.columns = ["Date","Names","close_price"]      


def returns_real_data(index,start_date,end_date):
    Prices = df_all_returns.iloc[list(df_all_returns['Names'] == list_names_unique[index])]
    True_False = []
    Dates_between = list(Prices['Date'] )
    for date in Dates_between:
        if start_date <= date <= end_date:
            True_False.append(True)
        else: 
            True_False.append(False)
    #take the range of date we are interested in 
    Prices_df = Prices.iloc[True_False]
    Prices_df.dropna(axis=1)
    Prices_list = list(Prices_df['close_price'])
    Returns_list = []
    for i in range(len(Prices_list)-1):
        Returns_list += [np.log(Prices_list[i+1]/Prices_list[i])]
    return(Returns_list)

def prices_real_data(index,start_date,end_date): #for jump diffusion process
    Prices = df_all_returns.iloc[list(df_all_returns['Names'] == list_names_unique[index])]
    True_False = []
    Dates_between = list(Prices['Date'] )
    for date in Dates_between:
        if start_date <= date <= end_date:
            True_False.append(True)
        else: 
            True_False.append(False)
    #take the range of date we are interested in 
    Prices_df = Prices.iloc[True_False]
    Prices_df.dropna(axis=1)
    Prices_list = list(Prices_df['close_price'])
    return(Prices_list)

list_names_unique = sorted(list(set(list(df_all_returns['Names']))))

data = pd.read_excel(
     os.path.join("realisedvariancesourcered.xlsx"),#with closing prices to generate from the same model
     engine='openpyxl',
)
data.describe()
list_dates_formated = []
list_dates = list(data['Unnamed: 0'])
list_names = list(data['Symbol'])
list_RV_5min = list(data['rv5'])
list_n = list(data['nobs'])
A_all = []
size_current = len(list_dates)
for i in range(0,size_current):
    date = list_dates[i].split(" ")[0].split("-")
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])    
    list_dates_formated.append(dt.datetime(year,month,day))
    A_all.append([dt.datetime(year,month,day),list_names[i],list_RV_5min[i],list_n[i]])
df_all = pd.DataFrame(A_all)
df_all.columns = ["Date","Names","RV_5min","n"]      



__Jump Diffusion model - Simulation__

In [ ]:
def merton_jump_paths(S, T, r, sigma,  lam, m, v, steps, Npaths):
    size=(steps,Npaths)
    delta = T/steps/365 
    poi_rv = np.multiply(np.random.poisson( lam*delta, size=size),
                         np.random.normal(m,v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r -  sigma**2/2 -lam*(m  + v**2*0.5))*delta +\
                              sigma*np.sqrt(delta) * \
                              np.random.normal(size=size)), axis=0)
    
    return 100*np.exp(geo+poi_rv)

def returns_merton_jump_paths(generated_path):
    generated_path_prices = [100]
    generated_path_returns = []
    n = int(steps/T)
    for i in range(T):
        generated_path_prices += [generated_path[i*n][0]]
        generated_path_returns += [math.copysign(1, np.log(generated_path_prices[i+1]/generated_path_prices[i]))*min(abs(np.log(generated_path_prices[i+1]/generated_path_prices[i])),0.2)]
    return(generated_path_returns)


__Heston model - Simulation__

In [ ]:
def HeMC (params):
    mu, v0, rho, kappa, theta, xi = params[0],  params[1],  params[2],  params[3],  params[4],  params[5]
    # Generate a Monte Carlo simulation for the Heston model
    S0 = 100
    # Generate random Brownian Motion
    MU  = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W   = np.random.multivariate_normal(MU, COV, T)
    W_S = W[:,0]
    W_v = W[:,1]
    delta = 1/365 #365 days in a year 
    # Generate paths
    vt    = np.zeros(T)
    vt[0] = v0
    St    = np.zeros(T)
    St[0] = S0
    rt    = np.zeros(T-1)
    for t in range(1,T):
        vt[t] = max(np.abs(vt[t-1] + kappa*(theta-np.abs(vt[t-1]))*delta + xi*np.sqrt(np.abs(vt[t-1]))*W_v[t]*np.sqrt(delta)),0.0001)
        St[t] = St[t-1]*np.exp((mu - 0.5*vt[t-1])*delta + np.sqrt(vt[t-1]*delta)*W_S[t])
        rt[t-1] =math.copysign(1,mu - 0.5*vt[t-1]*delta + np.sqrt(vt[t-1]*delta)*W_S[t])*min(abs(mu - 0.5*vt[t-1]*delta + np.sqrt(vt[t-1]*delta)*W_S[t]),0.5)

    return rt


__Rough Heston model - Simulation__

In [ ]:
def simulation_Heston(params,r_abs_max = 0.5,T = T, N = N):
    n = N/T*365 #to convert T in years
    rho, Lambda, theta,nu, H, v0 = params[0], params[1], params[2], params[3], params[4], params[5]
    # Generate random Brownian Motion
    MU  = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W   = np.random.multivariate_normal(MU, COV, N+1)
    W_S = W[:,0]
    W_v = W[:,1]
    # Generate paths
    var = np.zeros(N+1)
    var[0] = v0
    rt    = np.zeros(N)
    r_list = []
    sigma_list = []
    r = 0 
    for i in range(1,N):
        SUM1 = 0
        SUM2 = 0
        for j in range(1,i):
            SUM1 += 1/(H+0.5)*(theta - var[j])*(-(i/(n) - j/n)**(H+0.5)+(i/n - (j +1)/n)**(H+0.5))
            SUM2 += ((i/n - (j-0.5)/n)**(H-0.5))*np.sqrt(var[j])*(W_v[j+1]-W_v[j])*np.sqrt(1/n)
        var[i+1] = abs(v0 + Lambda/float(gamma(H+0.5))*SUM1+nu/float(gamma(H+0.5))*SUM2)
        rt[i] = np.sqrt(var[i+1])*W_S[i+1]*np.sqrt(1/n)
        r += rt[i]
        if (i%int(n/365) == 0):
            r = math.copysign(min(r_abs_max,abs(r)),r)
            r_list.append(r)
            r = 0 
            sigma_list.append(np.sqrt(var[i+1]))
    return(r_list)


__Rough fSV model - Simulation__

In [ ]:
def data_r_generator_opt(nu,Lambda,ksi,H,fbm_sample):
    Var_Y_t = (nu**2)/(2*Lambda**(2*H))*float(gamma(1+2*H))
    eta = np.log(ksi) - 0.5*Var_Y_t
    Y0 = norm.rvs(loc=eta, scale=1, size=1)
    Y_list = [Y0]
    Y = Y0
    Delta = T/N/365 
    n = 1/Delta
    for i in range(0,N):
        Y = eta + (Y - eta)*np.exp(-Lambda*Delta)+nu*np.exp(-Lambda*Delta/2)*(fbm_sample[i+1]-fbm_sample[i])
        Y_list.append(Y)
    sigma_list = []
    r_list = []
    I_list = []#list of increments
    r = 0
    for i in range(0,N):
        sigma = np.sqrt(np.exp(Y_list[i])) 
        I = np.random.randn(1)[0]*np.sqrt(1/n)
        I_list.append(I)
        r += sigma[0]*I
        if (i%int(n/365) == 0):
            r_list.append(r)
            r = 0 
            sigma_list.append(sigma[0])
    return(r_list)

def data_S_generator_opt(X):
    l = len(X)
    S = [100]
    s = 100
    for i in range(l):
        s = s*(1+X[i])
        S.append(s)
    return(S)


__Tools for parameters estimation__

In [ ]:
def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + dt.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=rd.randint(0, int((end - start).total_seconds())),
    )

N_iteration = 50
Number = 20
Number_Heston = 10
N_iteration_Rough_Heston = 10
N_iteration_Rough_fSV = 10
N_iteration_Heston = 10

__Jump Diffusion model - Estimation parameters__

In [ ]:
r=0.02
def TS_mean(params): 
    sigma,  lam, m, v = params[0], params[1], params[2], params[3]
    y = r - (sigma**2)/2-lam*(np.exp(m+0.5*(v**2))-1)+lam*m
    return(y)

def TS_var(params):
    sigma,  lam, m, v = params[0], params[1], params[2], params[3]
    y = (sigma**2) + lam*(v**2)+ lam*(m**2)
    return(y)

def TS_skew(params):
    sigma,  lam, m, v = params[0], params[1], params[2], params[3]
    y =(lam*(3*(v**2)*m+m**3))/((((sigma**2) + lam*(v**2)+ lam*(m**2)))**1.5)
    return(y)

def TS_kurt(params):
    sigma,  lam, m, v = params[0], params[1], params[2], params[3]
    y = (lam*(3*(v**4)+6*(v**2)*(m**2)+m**4))/((((sigma**2) + lam*(v**2)+ lam*(m**2)))**2)
    return(y)

def optim(params_real):
    mean_real, var_real, skew_real, kurt_real = params_real[0], params_real[1], params_real[2], params_real[3]
    def f(params):
        y = 2*abs(TS_mean(params)-mean_real)+2*abs(TS_var(params)-var_real)+abs(TS_skew(params)-skew_real)+abs(TS_kurt(params)-kurt_real)
        return(y)
    sigma0 = rd.uniform(0.02,0.1)
    lam0 = rd.uniform(0.0001,1)
    m0 = rd.uniform(-1,1)
    v0 =  rd.uniform(0.02,0.5)
    params0 = np.array([sigma0,lam0,m0,v0]) 
    res = optimize.minimize(f, params0, tol=1e-1,bounds = [(-5,5),(0,5),(-5,5),(0,5)])
    params_opt = res.x 
    return(params_opt)

def Jump_Diffusion_parameters_estimation(N_iteration,index):
    print(list_names_unique[index])
    params_fitted = np.array([0,0,0,0],dtype = float)
    for i in range(N_iteration):
        TIME = 2*T
        end_date = dt.datetime(2020,1,1) - dt.timedelta(TIME)
        start_date = random_date(dt.datetime(2000,1,1),end_date)
        end_date = start_date + dt.timedelta(TIME)
        end_date_final = random_date(end_date,dt.datetime(2021,1,1))
        prices = prices_real_data(index,start_date,end_date_final)
        prices_mean = np.mean(prices)
        prices_var = np.var(prices)
        prices_skew = skew(prices)
        prices_kurt = kurtosis(prices)
        params_real = np.array([prices_mean,prices_var,prices_skew,prices_kurt])
        params_fitted += optim(params_real)
    params_fitted = params_fitted/N_iteration
    return(params_fitted)



__Heston model - Estimation parameters__

In [ ]:
def moments_1(returns_list):
    return(stats.mean(returns_list))

def moments_2(returns_list):
    return(stats.variance(returns_list))

def moments_3(returns_list):
    return(skew(returns_list))

def moments_4(returns_list):
    return(kurtosis(returns_list))

def moments_5(returns_list):
    return(moment(returns_list,5))

def moments_6(returns_list):
    return(moment(returns_list,6))

def moment_vectors(returns_list):
    M1 = moments_1(returns_list)
    M2 = moments_2(returns_list)
    M3 = moments_3(returns_list)
    M4 = moments_4(returns_list)
    M5 = moments_5(returns_list)
    M6 = moments_6(returns_list)
    return([M1,M2,M3,M4,M5,M6])

def N_simulations_moments_Heston(Number,params, T =T, N=N):
    moments = np.array([[0]*6]*Number, dtype=float)
    for i in range(Number):
        returns_list = HeMC(params)
        M = moment_vectors(returns_list)
        moments[i,] = M
    return(moments)

def global_loss_L1_Heston(params,index,Number = Number_Heston):
    sim_moments = N_simulations_moments_Heston(Number,params)
    n_sim_returns = sim_moments.shape[0]#number of lines
    n_moments = sim_moments.shape[1] 
    moments_losses = np.array([[0]*6]*n_sim_returns, dtype=float)
    for i in range(n_sim_returns):
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2021,1,1))
        TIME = 2*T
        end_date = start_date + dt.timedelta(TIME)
        returns = returns_real_data(index,start_date,end_date)
        real_moment = moment_vectors(returns)
        for j in range(n_moments):
            moments_losses[i,j] = abs(sim_moments[i,j]-real_moment[j])
    vector_losses = np.mean(moments_losses, axis=0)
    vector_losses_norm = np.array([0]*n_moments, dtype=float)
    for j in range(n_moments):
        vector_losses_norm[j] = vector_losses[j]/np.max(moments_losses[:,j])#could have scaled with the sum
    mean_losses_norm = np.mean(vector_losses_norm)
    mean_losses = np.mean(vector_losses)
    return(mean_losses_norm)

def Heston_parameters_estimation(N_iteration_Heston,index):
    print(list_names_unique[index])
    params_fitted = 0
    for i in range(N_iteration_Heston): 
        mu, v0, rho, kappa, theta, xi = rd.uniform(0.02,0.2),rd.uniform(0.01,0.5), rd.uniform(-0.7,0.1), rd.uniform(0.1,0.5), rd.uniform(0.05,0.4), rd.uniform(0.1,0.35)
        x0 = np.array([mu, v0, rho, kappa, theta, xi],dtype = float)
        res = optimize.minimize(global_loss_L1_Heston,x0,args=(index,Number),method = "powell", bounds = [(0.00001,2),(0.0001,1),(-1,1),(0.000001,10),(0.0001,10),(0.000001,10)])
        params_fitted += res.x
    params_fitted = params_fitted/N_iteration_Heston
    return(params_fitted)

__Rough Heston model - Estimation parameters__

In [ ]:
def moments_1(returns_list):
    return(stats.mean(returns_list))

def moments_2(returns_list):
    return(stats.variance(returns_list))

def moments_3(returns_list):
    return(skew(returns_list))

def moments_4(returns_list):
    return(kurtosis(returns_list))

def moments_5(returns_list):
    return(moment(returns_list,5))

def moments_6(returns_list):
    return(moment(returns_list,6))

def moment_vectors(returns_list):
    M1 = moments_1(returns_list)
    M2 = moments_2(returns_list)
    M3 = moments_3(returns_list)
    M4 = moments_4(returns_list)
    M5 = moments_5(returns_list)
    M6 = moments_6(returns_list)
    return([M1,M2,M3,M4,M5,M6])

def N_simulations_moments(Number,params, T =T, N=N):
    moments = np.array([[0]*6]*Number, dtype=float)
    for i in range(Number):
        returns_list = simulation_Heston(params,0.5,T , N )
        M = moment_vectors(returns_list)
        moments[i,] = M
    return(moments)

def global_loss_L1(params,index,Number = Number,T =T, N=N):
    sim_moments = N_simulations_moments(Number,params,T,N)
    n_sim_returns = sim_moments.shape[0]#number of lines
    n_moments = sim_moments.shape[1] 
    moments_losses = np.array([[0]*6]*n_sim_returns, dtype=float)
    for i in range(n_sim_returns):
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2021,1,1))
        TIME = 2*T
        end_date = start_date + dt.timedelta(TIME)
        returns = returns_real_data(index,start_date,end_date)
        real_moment = moment_vectors(returns)
        for j in range(n_moments):
            moments_losses[i,j] = abs(sim_moments[i,j]-real_moment[j])
    vector_losses = np.mean(moments_losses, axis=0)
    vector_losses_norm = np.array([0]*n_moments, dtype=float)
    for j in range(n_moments):
        vector_losses_norm[j] = vector_losses[j]/np.max(moments_losses[:,j])#could have scaled with the sum
    mean_losses_norm = np.mean(vector_losses_norm)
    mean_losses = np.mean(vector_losses)
    return(mean_losses_norm)



def Rough_Heston_parameters_estimation(N_iteration_Rough_Heston,index):
    print(list_names_unique[index])
    params_fitted = 0
    for i in range(N_iteration_Rough_Heston):
        v0, rho, Lambda, theta,nu, H = rd.uniform(0.02,0.2),rd.uniform(-0.7,0.1), rd.uniform(0.3,1), rd.uniform(0.02,0.2), rd.uniform(0.1,0.5), rd.uniform(0.05,0.25)
        x0 = np.array([rho, Lambda, theta,nu, H, v0],dtype = float)
        res = optimize.minimize(global_loss_L1,x0,args=(index,Number),method = "powell", bounds = [(-1,1),(0.001,10),(0,2),(-10,10),(0,0.5),(0,2)])
        params_fitted += res.x
    params_fitted = params_fitted/N_iteration_Rough_Heston
    return(params_fitted)

__Rough fSV model - Estimation parameters__

In [ ]:
def create_RV_5min_list(index,start_date,end_date):
    RV = df_all.iloc[list(df_all['Names'] == list_names_unique[index])]
    True_False = []
    Dates_between = list(RV['Date'] )
    for date in Dates_between:
        if start_date <= date <= end_date:
            True_False.append(True)
        else: 
            True_False.append(False)
    RV_df = RV.iloc[True_False]
    RV_df.dropna(axis=1)
    RV_list = list(RV_df['RV_5min'])
    n = int(RV_df['n'].mean())
    return(RV_list,n)

def Kappa(l,xi,Lambda,nu,H):
    kappa0 = nu**2/(2*(Lambda**(2*H)))*float(gamma(1+2*H))
    if l == 0:
        y = kappa0
    else:
        y = kappa0*math.cosh(Lambda*l)-(nu**2)*(abs(l)**(2*H))/2*mpmath.hyp1f2(1,H+0.5,H+1,(Lambda**2)*(l**2)/4) 
    return(float(y))

def G(l,xi,Lambda,nu,H):
    C = (nu**2)/((2*H+1)*(2*H+2))
    if l == 0:
        y = (xi**2)*np.exp(Kappa(0,xi,Lambda,nu,H))*(1-Kappa(0,xi,Lambda,nu,H)+2*Kappa(0,xi,Lambda,nu,H)/(Lambda**2)*(math.cosh(Lambda)-1)- C*mpmath.hyp1f2(1,H+1.5,H+2,(Lambda**2)/4))
    else:
         y = (xi**2)*np.exp(Kappa(l,xi,Lambda,nu,H))*(1-Kappa(l,xi,Lambda,nu,H)+2*Kappa(0,xi,Lambda,nu,H)/(Lambda**2)*math.cosh(Lambda*l)*(math.cosh(Lambda)-1)
                                                    - (xi**2)*np.exp(Kappa(l,xi,Lambda,nu,H))*C/2*((l+1)**(2*H+2))*mpmath.hyp1f2(1,H+1.5,H+2,(Lambda**2)*((l+1)**2)/4)
                                                    -(xi**2)*np.exp(Kappa(l,xi,Lambda,nu,H))*C/2*((l-1)**(2*H+2))*mpmath.hyp1f2(1,H+1.5,H+2,(Lambda**2)*((l-1)**2)/4)
                                                    + (xi**2)*np.exp(Kappa(l,xi,Lambda,nu,H))*C*(l**(2*H+2))*mpmath.hyp1f2(1,H+1.5,H+2,(Lambda**2)*(l**2)/4))   
    return(float(y))

def c(xi,Lambda,nu,H,n):#function of function
    def to_integrate(u):
        y = (1-u)*np.exp(Kappa(u/n,xi,Lambda,nu,H))
        return(y)
    y = 4*(xi**2)/n*quad(to_integrate,0,1)[0]
    return(y)

def gc(l,xi,Lambda,nu,H,n):
    if l == 2:
        y = G(l,xi,Lambda,nu,H) + c(xi,Lambda,nu,H,n)
    else:
        y = G(l,xi,Lambda,nu,H)
    return(y)

def Gc_vector(k,xi,Lambda,nu,H,n):
    L = [xi]
    for i in range(0,k+1):
        L.append(gc(i,xi,Lambda,nu,H,n))
    y = np.array(L,dtype=float)
    y = np.reshape(y,(k+2,1))
    return(y)

def hat_IV_vector(t,RV_list,n,k):
    L = [RV_list[t-1]]*min(k+2,t+2)+[0]*max((k-t),0)
    a = np.array(L,dtype=float)
    LB = [1,RV_list[t-1]]
    u = t - 1
    i = 1
    while (i != 5):
        if (u != 0):
            u = u - 1 
            i +=1 
            LB.append(RV_list[u]) 
        else:
            i = 5
    b = np.array(LB+[0]*max((k-t+1),0),dtype=float)
    y = a*b 
    y = np.reshape(y,(k+2,1))
    return(y)

def hat_m(xi,Lambda,nu,H,RV_list,n,k):
    T = len(RV_list)
    y = np.reshape(np.array((k+2)*[0],dtype=float),(k+2,1))
    for i in range(1,T):
        y = y + hat_IV_vector(i,RV_list,n,k)
    y = y/T
    y = y - Gc_vector(k,xi,Lambda,nu,H,n)
    return(y)

def function_to_min(theta,RV_list,n,k):
    xi,Lambda,nu,H = theta[0],theta[1],theta[2],theta[3]
    T = len(RV_list)
    Product = np.reshape(hat_m(xi,Lambda,nu,H,RV_list,n,k),(1,k+2)).dot(np.eye(k+2)).dot(np.reshape(hat_m(xi,Lambda,nu,H,RV_list,n,k),(k+2,1)))
    return(Product[0][0])

def hat_theta_one(initial_guess,RV_list,n,k=4,B=[(0.00001,0.1),(0.000001,0.001),(0.001,4),(0.01,0.5)]):
    res = optimize.minimize(function_to_min,initial_guess,args =(RV_list,n,k),bounds=B)
    if res.success:
        fitted_params = res.x
        return(fitted_params)
    else:
        print('error')#raise ValueError()
        
def W_hat(theta,RV_list,n,k):
    xi,Lambda,nu,H = theta[0],theta[1],theta[2],theta[3]
    T = len(RV_list)
    y = np.array([(k+2)*[0]]*(k+2),dtype=float)
    for i in range(1,T+1): 
        y = y + np.reshape(hat_IV_vector(i,RV_list,n,k)- T*Gc_vector(k,xi,Lambda,nu,H,n),(k+2,1)).dot(np.reshape(hat_IV_vector(i,RV_list,n,k)- T*Gc_vector(k,xi,Lambda,nu,H,n),(1,k+2))) 
    y = y/T
    y = np.linalg.inv(y)
    return(y)

def function_to_min_2(theta,W,RV_list,n,k=4):
    xi,Lambda,nu,H = theta[0],theta[1],theta[2],theta[3]
    T = len(RV_list)
    Product = np.reshape(hat_m(xi,Lambda,nu,H,RV_list,n,k),(1,k+2)).dot(W).dot(np.reshape(hat_m(xi,Lambda,nu,H,RV_list,n,k),(k+2,1)))#T c'est le k 
    return(Product[0][0])

def hat_theta_two(initial_guess,theta_fitted1,RV_list,n,k=4,B=[(0.00001,0.1),(0.000001,0.001),(0.001,4),(0.01,0.5)]):
    W = W_hat(theta_fitted1,RV_list,n,k)
    res = optimize.minimize(function_to_min_2,initial_guess,args =(W,RV_list,n,k),bounds=B)
    if res.success:
        fitted_params = res.x
        return(fitted_params)
    else:
        raise ValueError()

def globalizer(i,theta0 = [0.2,0.0004,1,0.08],k=4,B=[(0.000001,1),(0.0000001,0.01),(0.0001,4),(0.005,0.5)]):
    start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2021,1,1))
    end_date = random_date(start_date,dt.datetime(2021,1,2))
    RV_list, n = create_RV_5min_list(i,start_date,end_date)
    fitted_1 = hat_theta_one(theta0,RV_list,n,k,B)
    print('$hat{theta_{1}}=$',fitted_1)
    #fitted_2 = hat_theta_two(fitted_1,fitted_1,RV_list,n,k,B)
    #print('$hat{theta_{2}}=$',fitted_2)
    return(fitted_1)

def Rough_fSV_parameters_estimation(N_iteration_Rough_fSV,index):
    print(list_names_unique[index])
    params_fitted = 0
    for i in range(N_iteration_Rough_fSV):
        xi,Lambda,nu,H = rd.uniform(0.001,0.3),rd.uniform(0.0004,0.001),rd.uniform(0.1,2),rd.uniform(0.04,0.15)
        theta0 = [xi,Lambda,nu,H]
        res = globalizer(index,theta0 ,4,[(0.000001,1),(0.0000001,0.2),(0.00001,4),(0.0005,0.5)])
        params_fitted += res
    params_fitted = params_fitted/N_iteration_Rough_fSV
    return(params_fitted)

__Parameters estimation for the 4 models__ 

__Comparison - Loss function__

In [ ]:
def losses_Jump_Diffusion(index,N_simulation):
    Jump_Diffusion_parameters = Jump_Diffusion_parameters_estimation(N_iteration,index)
    sigma,  lam, m, v = Jump_Diffusion_parameters
    L_Jump_Diffusion = 0
    L = 0
    for i in range(N_simulation): 
        Jump_Diffusion_returns = returns_merton_jump_paths(merton_jump_paths(100, T, 0.02, sigma,  lam, m, v, N, 1))
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2020,1,1))
        TIME = 3*T
        end_date = start_date + dt.timedelta(days = TIME)
        Real_returns = returns_real_data(index,start_date,end_date)
        Jump_Diffusion_prices = data_S_generator_opt(Jump_Diffusion_returns)
        Real_prices = data_S_generator_opt(Real_returns)
        for j in range(T):
            L_Jump_Diffusion += abs((Real_prices[j] - Jump_Diffusion_prices[j]))
        L_Jump_Diffusion = L_Jump_Diffusion/T
        L +=  L_Jump_Diffusion
        L_Jump_Diffusion = 0
    L = L/N_simulation
    print("The parameters of the Jump Diffusion model for the equity "+str(list_names_unique[index])+" are ("
          +str(Jump_Diffusion_parameters[0])+","+str(Jump_Diffusion_parameters[1])+","+str(Jump_Diffusion_parameters[2])
          +","+str(Jump_Diffusion_parameters[3])+").")
    return(L)

def losses_Heston(index,N_simulation):
    Heston_parameters = Heston_parameters_estimation(N_iteration_Heston,index)
    L_Heston = 0
    L = 0
    for i in range(N_simulation): 
        Heston_returns = HeMC (Heston_parameters)
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2020,1,1))
        TIME = 3*T
        end_date = start_date + dt.timedelta(days = TIME)
        Real_returns = returns_real_data(index,start_date,end_date)
        Heston_prices = data_S_generator_opt(Heston_returns)
        Real_prices = data_S_generator_opt(Real_returns)
        for j in range(T):
            L_Heston += abs((Real_prices[j] - Heston_prices[j]))
        L_Heston =  L_Heston/T
        L += L_Heston
        L_Heston = 0
    L = L/N_simulation
    print("The parameters of the Heston model for the equity "+str(list_names_unique[index])+" are ("
          +str(Heston_parameters[0])+","+str(Heston_parameters[1])+","+ str(Heston_parameters[2])
          +","+str(Heston_parameters[3])+","+str(Heston_parameters[4])+","+str(Heston_parameters[5])
          +").")
    return(L)

def losses_Rough_Heston(index,N_simulation):
    Rough_Heston_parameters = Rough_Heston_parameters_estimation(N_iteration_Rough_Heston,index)
    L_Rough_Heston = 0
    L = 0
    for i in range(N_simulation): 
        Rough_Heston_returns = simulation_Heston(Rough_Heston_parameters,r_abs_max = 0.5,T = T, N = N)
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2020,1,1))
        TIME = 3*T
        end_date = start_date + dt.timedelta(days = TIME)
        Real_returns = returns_real_data(index,start_date,end_date)
        Rough_Heston_prices = data_S_generator_opt(Rough_Heston_returns)
        Real_prices = data_S_generator_opt(Real_returns)
        for j in range(T):
            L_Rough_Heston += abs((Real_prices[j] - Rough_Heston_prices[j]))
        L_Rough_Heston  = L_Rough_Heston/T
        L += L_Rough_Heston
        L_Rough_Heston = 0
    L = L/N_simulation
    print("The parameters of the Rough Heston model for the equity "+str(list_names_unique[index])+" are ("
          +str(Rough_Heston_parameters[0])+","+str(Rough_Heston_parameters[1])+","+str(Rough_Heston_parameters[2])
          +","+str(Rough_Heston_parameters[3])+","+str(Rough_Heston_parameters[4])+","+str(Rough_Heston_parameters[5])
          +").") 
    return( L)

def losses_Rough_fSV(index,N_simulation):
    Rough_fSV_parameters = Rough_fSV_parameters_estimation(N_iteration_Rough_fSV,index)
    L_Rough_fSV = 0
    L = 0
    ksi,Lambda,nu,H = Rough_fSV_parameters[0],Rough_fSV_parameters[1],Rough_fSV_parameters[2],Rough_fSV_parameters[3]
    for i in range(N_simulation): 
        f = FBM(n=N, hurst=H, length=T, method='cholesky')
        fbm_sample = f.fbm()
        Rough_fSV_returns = data_r_generator_opt(nu,Lambda,ksi,H,fbm_sample)
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2020,1,1))
        TIME = 3*T
        end_date = start_date + dt.timedelta(days = TIME)
        Real_returns = returns_real_data(index,start_date,end_date)
        Rough_fSV_prices = data_S_generator_opt(Rough_fSV_returns)
        Real_prices = data_S_generator_opt(Real_returns)
        for j in range(T):
            L_Rough_fSV += abs((Real_prices[j] - Rough_fSV_prices[j]))
        L_Rough_fSV  = L_Rough_fSV/T  
        L += L_Rough_fSV 
        L_Rough_fSV  = 0
    L = L/N_simulation
    print("The parameters of the Rough fSV model for the equity "+str(list_names_unique[index])+" are ("
          +str(Rough_fSV_parameters[0])+","+str(Rough_fSV_parameters[1])+","+str(Rough_fSV_parameters[2])
          +","+str(Rough_fSV_parameters[3])+").") 
    return( L)

def losses_constant_initial_prices(index,N_simulation):
    L_constant = 0
    L = 0
    for i in range(N_simulation):  
        start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2020,1,1))
        TIME = 3*T
        end_date = start_date + dt.timedelta(days = TIME)
        Real_returns = returns_real_data(index,start_date,end_date)
        prices = np.array([100]*(T),dtype=float)
        Real_prices = data_S_generator_opt(Real_returns)
        for j in range(T):
            L_constant += abs((Real_prices[j] - prices[j]))
        L_constant  = L_constant/T  
        L += L_constant 
        L_constant  = 0
    L = L/N_simulation
    return( L)
    
    
    

__Example of use__

In [ ]:
start= timer()
L_AORD_Jump_Diffusion = losses_Jump_Diffusion(1,10) 
end = timer()

print("Time taken:", end-start)

In [ ]:
def one_path_simulations(index,params_JD, params_H, params_RH, params_fSV, T, N):
    rT_RH = simulation_Heston(params_RH,r_abs_max = 0.5,T = T, N = N)
    nu,Lambda,ksi,H = params_fSV[2], params_fSV[1], params_fSV[0], params_fSV[3]
    f = FBM(n=N, hurst=H, length=T, method='cholesky')
    fbm_sample = f.fbm()
    rT_fSV_10 = data_r_generator_opt(nu,Lambda,ksi,H,fbm_sample)
    rT_H = HeMC (params_H)
    sigma,  lam, m, v = params_JD[0], params_JD[1], params_JD[2], params_JD[3]
    rT_JD =  returns_merton_jump_paths(merton_jump_paths(100, T, 0.02, sigma,  lam, m, v, N, 1))
    start_date = random_date(dt.datetime(2000,1,1),dt.datetime(2019,1,1))
    TIME = 2*T
    end_date = start_date + dt.timedelta(days = TIME)
    rT_real = returns_real_data(index,start_date,end_date)
    rT_real = rT_real[:len(rT_H)+1]
    ST_real =  data_S_generator_opt(rT_real) 
    ST_H = data_S_generator_opt(rT_H)
    ST_fSV_10 = data_S_generator_opt(rT_fSV_10)
    ST_RH = data_S_generator_opt(rT_RH)
    ST_JD = data_S_generator_opt(rT_JD)
    return((rT_real, rT_H, rT_fSV_10, rT_RH, rT_JD),(ST_real, ST_H, ST_fSV_10, ST_RH, ST_JD))

In [ ]:
def plot_4sims_and_real(R,S,save ='no',name = "num_index"):
    rT_real, rT_H, rT_fSV_10, rT_RH, rT_JD = R
    ST_real, ST_H, ST_fSV_10, ST_RH, ST_JD = S
    plt.figure()
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(15, 6)
    X = range(T)
    ax1.plot(X,ST_real[:T],linewidth=0.8)
    ax1.plot(X,ST_fSV_10[:T],linewidth=0.8)
    ax1.plot(X,ST_RH[:T],linewidth=0.8)
    ax1.plot(X,ST_JD[:T],linewidth=0.8)
    ax1.set(xlabel='Number of days elapsed t since the starting date\n \n \n', ylabel='Daily prices '+ r'$S_{t}$',title="Plot of the price time series against time\n with an initial price "+r"$S_{0}=100$")
    ax2.plot(rT_real,linewidth=0.4,label ='Real data')
    ax2.plot(rT_fSV_10,linewidth=0.4,label ='Rough fSV model')
    ax2.plot(rT_RH,linewidth=0.4,label ='Rough Heston model')
    ax2.plot(rT_JD,linewidth=0.4,label ='Jump diffusion model')
    ax2.set(xlabel='Number of days elapsed t since the starting date \n \n \n', ylabel='Daily returns '+ r'$r_{t}$',title="Plot of the return time series against time" )
    leg = fig.legend(frameon=True, loc='lower center', ncol=5)
    # get the individual lines inside legend and set line width
    for line in leg.get_lines():
        line.set_linewidth(4)
    if save != 'no':
        plt.savefig('Comparison_'+str(name)+'.pdf', dpi=300, bbox_inches='tight')


__Example of use__

In [ ]:
params_JD_SPX = np.array([ 1.2902517051852678,1.9326961807012852,-0.64727243732107,0.37134788480208913])
params_H_SPX = np.array([0.8904757261820736,0.5073357551034949,-0.2371919515133079,5.317697836966241,5.27149187746957,4.893097016530513]) 
params_RH_SPX = np.array([0.06173884810561812,5.81688835552416,0.8585671402167568,-1.1499604689838576,0.24471108690569632,0.9189124895509331]) 
params_fSV_SPX = np.array([5.394818894781241e-05,0.009999999999999273,0.48141362516206093,0.034625746342434933]) 

T = 20
N = 6*T
R_SPX,S_SPX = one_path_simulations(26, params_JD_SPX, params_H_SPX, params_RH_SPX, params_fSV_SPX, T, N)

In [ ]:
plot_4sims_and_real(R_SPX,S_SPX,save ='no',name = "num_index")

__Implementation of the Brownian Motion for the classifier (Chapter 3 Thesis)__

In [ ]:
def brownian_motion(N, T, h):
    """
    Simulates a Brownian motion
    :param int N : the number of discrete steps
    :param int T: the number of continuous time steps
    :param float h: the variance of the increments
    """
    dt = 1. * T/N  # the normalizing constant
    random_increments = np.random.normal(0.0, 1.0 * h, N)*np.sqrt(dt)  # the epsilon values
    brownian_motion = np.cumsum(random_increments)  # calculate the brownian motion
    brownian_motion = np.insert(brownian_motion, 0, 0.0) # insert the initial condition

    return brownian_motion, random_increments
